# Synthetic Data on Healthcare

This notebook is a demonstration on how to use SQLSynthGen for generating synthetic data, for a healthcare use case.

## Initial Setup

In [ ]:
from os import chdir

HOME_DIRECTORY: str = "/home/jovyan"

chdir(HOME_DIRECTORY)

Let's try our package:

In [ ]:
from lib.healthcare_demo import say_hello

say_hello()